In [8]:
import pandas as pd
import numpy as np

In [9]:
#Importación de la data en los dataframes.
df_amazon = pd.read_csv('./Datasets/amazon_prime_titles.csv')
df_disney_plus = pd.read_csv('./Datasets/disney_plus_titles.csv')
df_hulu = pd.read_csv('./Datasets/hulu_titles.csv')
df_netflix = pd.read_json('./Datasets/netflix_titles.json')

#Se agrega la columan "plataforma" a cada dataframe.
dataframes = [df_amazon, df_disney_plus, df_hulu, df_netflix]
plataformas = ['amazon', 'disney plus','hulu','netflix']
plataformas_numero = ['0','1','2','3']
for number, dataframe in enumerate(dataframes):
    dataframe['plataform']=plataformas[number]  #Se agregan los datos de plataformas a la columna plataform.
    dataframe['plataform_id']=plataformas_numero[number] #Se agrega una columna con id para la plataforma.

#Creación del dataframe unificado
df = pd.DataFrame() #Creación de un dataframe vacío donde irán unidos todos los dataframes de las películas.
df = pd.concat([dataframe for dataframe in dataframes], ignore_index=True) #Se unen los dataFrames en uno solo.
df['show_id']=df['show_id'] + df['plataform_id'] #Se genera un id subrogado con el show_id y plataform_id

#Limpieza de las columnas duplicados y Columnas no necesarias
df.drop_duplicates(inplace=True) #Se limpia el dataframe de posibles duplicados (Los duplicados se consideran si son filas iguales).
df.drop(columns=['country', 'date_added','rating', 'description'], inplace=True) #Se eliminan columnas que no se utilizan

#Cambio de la columna "duration"
df_dur=df["duration"].str.split(expand=True) #Se crea un dataframe con los datos de la columna "duration" separando el número de la unidad
df_dur.rename(columns={0:"duration",1:"type (min-seasons)"},inplace=True) #Se renombran las columnas.
df.drop(columns='duration', inplace=True) #Se elimina la columna 'duration' del dataframe original
df['duration'] = df_dur["duration"] #Se agrega la columna duration con el número
df['type (min-seasons)'] = df_dur['type (min-seasons)'] #se agrega la columna type (min-Seasons)
df['duration'].fillna(0, inplace=True)
df.fillna('sin dato', inplace=True) #Se llenan los datos vacíos como: sin dato.
df["duration"]=df["duration"].astype("int64") #Se cambia el tipo de dato por int en la columna duration

#Promedios de duración de películas y series por plataforma
prom_movies = {}
prom_series = {}
for plataform in df['plataform'].unique():
    prom_movies[plataform] = round(df[(df['duration'] !=0 ) & (df['type']=='Movie')&(df['plataform']==plataform)]['duration'].mean()) #Se halla la media de la duración de películas por plataforma
    prom_series[plataform] = round(df[(df['duration'] !=0 ) & (df['type']=='TV Show') & (df['plataform']==plataform)]['duration'].mean()) #Se halla la media de la duración de series por plataforma
    df['duration'][df['duration'].loc[(df['duration'] == 0) & (df['type']=='Movie') & (df['plataform']==plataform)].index]=prom_movies[plataform] #Se guarda el
    df['duration'][df['duration'].loc[(df['duration'] == 0) & (df['type']=='TV Show') & (df['plataform']==plataform)].index]=prom_series[plataform]

#Se cambian los valores "sin dato" de las unidades de tiempo (minutos o seasons) en la columna correspondiente
df['type (min-seasons)'][df['type (min-seasons)'].loc[(df['type']=='TV Show')].index]='season'
df['type (min-seasons)'][df['type (min-seasons)'].loc[(df['type']=='Movie')].index] = 'min'

#Cambio de todos los textos a minusculas
df['type']=df['type'].str.lower()
df['title']=df['title'].str.lower()
df['director']=df['director'].str.lower()
df['listed_in']=df['listed_in'].str.lower()
df['cast']=df['cast'].str.lower()

#Se convierten los actores en listas dentro del dataframe
df['cast_1'] = df["cast"].str.split(",") #se separan por comas los actores dentro de un campo, se convierten en listas y se agregan como columna nueva
df.drop(columns=['cast'],inplace=True)   #Se elimina la columna cast para dejar solo la que los tiene como listas
df.rename(columns={'cast_1':'cast'}, inplace=True) #Se renombra la columna cast_1 que fue creada para introducir los actores

#Se convierten los géneros en listas dentro del dataframe al igual que con los actores
df['listed_in_1'] = df["listed_in"].str.split(",")
df.drop(columns=['listed_in'],inplace=True)
df.rename(columns={'listed_in_1':'listed_in'}, inplace=True)

#Creación de DataFrame para los actores
dic_actores = {'actor':[],
                'year':[],
                'plataform':[]} #Se crea un diccionario para luego ser convertido en dataframe
for index, actors in enumerate(df['cast']):
    for actor in actors:
        if len(actor)>3:
            dic_actores['actor'].append(actor) #Se agrega el actor al diccionario
            dic_actores['year'].append(df['release_year'][index]) #Se agrega el año correspondiente actor al diccionario
            dic_actores['plataform'].append(df['plataform'][index]) #Se agrega la plataforma correspondiente al actor al diccionario
df_actors = pd.DataFrame(dic_actores)
#df_actors=df_actors.drop_duplicates().reset_index()



#Creación de DataFrame para los generos
dic_genero = {'genero':[],
                'plataforma':[]} #Se crea un diccionario para luego ser convertido en dataframe
for index, generos in enumerate(df['listed_in']):
    for genero in generos:
        dic_genero['genero'].append(genero) #Se agrega el genero al diccionario
        dic_genero['plataforma'].append(df['plataform'][index])
df_genero = pd.DataFrame(dic_genero)
#df_genero = df_genero.drop_duplicates(ignore_index=True) #se borran los duplicados del data frame.

df_genero.to_csv('generos.csv')
df_actors.to_csv('actors.csv')
df.to_csv('dataframe.csv')

C:\Users\user\AppData\Local\Temp\ipykernel_16292\3268050869.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['duration'][df['duration'].loc[(df['duration'] == 0) & (df['type']=='Movie') & (df['plataform']==plataform)].index]=prom_movies[plataform] #Se guarda el
C:\Users\user\AppData\Local\Temp\ipykernel_16292\3268050869.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['duration'][df['duration'].loc[(df['duration'] == 0) & (df['type']=='TV Show') & (df['plataform']==plataform)].index]=prom_series[plataform]
C:\Users\user\AppData\Local\Temp\ipykernel_16292\3268050869.py:40: SettingWithCopyWarning: 
A value 

In [5]:
#Es posible usar la media para los datos NAN, porque:
mean_lists_movies = df[df['type'] == 'movie'][['plataform','release_year','duration']]
mean_lists_movies = mean_lists_movies.groupby(['plataform','release_year'])['duration'].max()
mean_lists_series= df.loc[df['type'] == 'tv show'][['plataform','release_year','duration']]
mean_lists_series = mean_lists_series.groupby(['plataform','release_year'])['duration'].max()

In [6]:
def get_max_duration(año, plataforma, tipo_film):
    message = f"Máximo de {plataforma} en {año} es"
    return {message:df[(df['release_year']==año)&(df['plataform']==plataforma)&(df['type (min-seasons)'] == tipo_film)]['duration'].max()}
get_max_duration(2000,'amazon','min')

{'Máximo de amazon en 2000 es': 177}

In [24]:
def get_count_plataform(plataforma):   
    return{f"Cantidad de películas en {plataforma} es":df[(df['plataform']==plataforma)&(df['type']=='movie')]['type'].count(),
    f"Cantidad de series en {plataforma} es":df[(df['plataform']==plataforma)&(df['type']=='tv show')]['type'].count()}
get_count_plataform('amazon')

{'Cantidad de películas en amazon es': 7814,
 'Cantidad de series en amazon es': 1854}

In [20]:
def get_listedin(genero):
    return {(df_genero[(df_genero["genero"]==genero)]["plataforma"].value_counts().idxmax()):
    (df_genero[(df_genero["genero"]==genero)]["plataforma"].value_counts().max())}

get_listedin('comedies')

#df_genero.head()

{'netflix': 1210}

In [260]:
def get_actor(plataforma, año):
    data = df_actors[(df_actors['year']==año)&(df_actors['plataform']==plataforma)]['actor']
    # return {data.value_counts().max()}
    if data.value_counts().index[0] == 'sin dato':
        return {data.value_counts().index[1]:data.value_counts()[1]}
    elif data.value_counts().index[0] != 'sin dato':
        return {data.value_counts().index[0]:data.value_counts()[0]}
get_actor('amazon',2020)

{' michael madsen': 7}